# TradingClient example

## Prepare client

In [1]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [2]:
# import the packages
import osmosispy
from osmosispy import pytypes
from osmosispy.trading_client import BinanceTradingClient, TradeData
import time

don't forget to set your API key and secret to the environment variables
`BINANCE_API_KEY` and `BINANCE_API_SECRET` respectively. Use [this article](https://www.binance.com/en/support/faq/how-to-create-api-360002502072) to get api keys

```bash
export BINANCE_API_KEY=qwerty
export BINANCE_API_SECRET=asdfgh
```

In [3]:
# Create a binance trading client
binance_client = BinanceTradingClient.from_env()

# # Alternatively, you can pass them to the client constructor.
# binance_client = BinanceTradingClient(
#     api_key="qwerty",
#     api_secret="asdfgh",
# )

## Get OSMO USDT trading info

In [6]:
fee = binance_client.get_trading_fee("OSMOUSDT")
# fee object will contain the following fields:
# maker_commision: float
# taker_commision: float

print(f"Trading fee: {fee}")

Trading fee: TradingFee(maker_commision=0.001, taker_commision=0.001)


In [7]:
price_in_usdt = binance_client.get_price(symbol = "OSMOUSDT")

print(f"Current price in USDT:\n{price_in_usdt}")

{'symbol': 'OSMOUSDT', 'price': '0.79000000'}
Current price in USDT:
denom: "USDT"
amount: "0.79000000"



## Compare OSMO prices on binance and Osmosis
Simple example of comparing OSMO prices on binance and Osmosis.

In [8]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import (
    query_pb2 as gamm_query,
    query_pb2_grpc as gamm_query_grpc,
)
from osmosispy import Sdk, Network, utils

In [9]:
# prepare the osmosis client
mnemonic_key = "satisfy adjust timber high purchase tuition stool faith fine install that you unaware feed domain license impose boss human eager hat rent enjoy dawn"

network = Network.mainnet()
trader = Sdk.authorize(key=mnemonic_key).with_network(network).with_config(pytypes.TxConfig())

stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)

# check the address
print(trader.address)

osmo1phaxpevm5wecex2jyaqty2a4v02qj7qmlmzk5a


In [10]:

stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
pool_raw_data = utils.deserialize(
    stub_gamm.Pool(
        # checking this pool https://app.osmosis.zone/pool/678
        gamm_query.QueryPoolRequest(pool_id=678)
    )
)

osmosis_pool = utils.deserialize_pool(pool_raw_data["pool"])

print(f"osmosis pool: {osmosis_pool}")

osmosis pool: {'address': 'osmo10venxtvdglryxkdmvjr8wa6n3ugja40rewddlxtg0pr30vmkf47sllgslg', 'id': 678, 'pool_params': {'swap_fee': 0.002, 'exit_fee': 0.0, 'smooth_weight_change_params': {'start_time': {'seconds': 0, 'nanos': 0}, 'duration': {'seconds': 0, 'nanos': 0}, 'initial_pool_weights': [], 'target_pool_weights': []}}, 'future_pool_governor': '24h', 'total_shares': {'denom': 'gamm/pool/678', 'amount': 1914364374789231521263681}, 'pool_assets': [{'token': {'denom': 'ibc/D189335C6E4A68B513C10AB227BF1C1D38C746766278BA3EEB4FB14124F1D858', 'amount': 8200195926141}, 'weight': 536870912000000}, {'token': {'denom': 'uosmo', 'amount': 10388969285133}, 'weight': 536870912000000}], 'total_weight': 1073741824000000}


In [11]:
def get_osmosis_trading_price(pool):
    price_data = utils.deserialize(
        stub_gamm.SpotPrice(
            gamm_query.QuerySpotPriceRequest(
                pool_id=pool["id"],
                base_asset_denom=pool["pool_assets"][0]["token"]["denom"],
                quote_asset_denom=pool["pool_assets"][1]["token"]["denom"],
            )
        )
    )

    return price_data["spot_price"]

# create trade callback
def trade_callback(trade: TradeData):
    print("===== OSMO USDT trades =====")
    print(f"binance price: {trade.price.amount}\nbinance quantity: {trade.quantity.amount}")
    
    osmosis_pool_price = get_osmosis_trading_price(osmosis_pool);
    print(f"osmosis price: {osmosis_pool_price}")

# start listening to trades
binance_client.listen_trades(cb=trade_callback, symbol="OSMOUSDT")

===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 51.00000000
osmosis price: 0.789317560000000000
===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 21.00000000
osmosis price: 0.789317560000000000
===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 16.35000000
osmosis price: 0.789317560000000000
===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 25.45000000
osmosis price: 0.789317560000000000


In [12]:
# listen for 60 seconds
time.sleep(60)

===== OSMO USDT trades =====
binance price: 0.79100000
binance quantity: 25.31000000
osmosis price: 0.789316590000000000


KeyboardInterrupt: 

===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 25.31000000
osmosis price: 0.789310220000000000
===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 67.51000000
osmosis price: 0.789310110000000000
===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 35.73000000
osmosis price: 0.789310110000000000
===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 71.59000000
osmosis price: 0.789302940000000000
===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 113.15000000
osmosis price: 0.789302940000000000
===== OSMO USDT trades =====
binance price: 0.79000000
binance quantity: 457.17000000
osmosis price: 0.789302940000000000
===== OSMO USDT trades =====
binance price: 0.79100000
binance quantity: 139.24000000
osmosis price: 0.789302940000000000
===== OSMO USDT trades =====
binance price: 0.79100000
binance quantity: 37.92000000
osmosis price: 0.789302950000000000
===== OSMO USDT trades =====


In [ ]:
# stop listening to trades
binance_client.stop_listeners()